# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** — проверка четырех гипотез:
1. На факт погашения кредита в срок влияет семейное положение
2. На факт погашения кредита в срок влияет наличие детей 
3. На факт погашения кредита в срок влияет уровень дохода
4. Оценка влияния цели кредита на его возврат в срок

**Ход исследования**

Данные о клиентах, которым выдан кредит находятся в файле `data.csv`. О качестве данных ничего не известно. Поэтому перед проверкой гипотез понадобится обзор данных. 

На этапе предобработки необходимо будет обработать пропуски, удалить дубликаты, заменить типы данных, выделить леммы с целями получения кредита и категоризировать данные.
 
Таким образом, исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
# чтение файла с данными и сохранение в переменную data
credit_clients = pd.read_csv('/datasets/data.csv')
# получение первых 10 строк таблицы data
display(credit_clients.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [3]:
# получение общей информации о данных в таблице data
credit_clients.info()
display(credit_clients.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


В таблице 12 столбцов.

Согласно документации к данным:
* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;  
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

**Замечания к данным:**
1. Пропуски в таблице в столбцах `days_employed` и `total_income`
2. Для целей исследования вещественный тип данных в этих столбцах избыточен, необходимо будет его поменять на целочисленный;
3. Данные в столбце `days_employed` содержат отрицательные и аномально большие значения
4. Минимальное значение по столбцу `children` = -1. Необходимо проверить число уникальных значений методом value_counts() и оценить данные (вероятно таким образом отмечали отсутствие информации по детям)
5. Минимальное значение по столбцу `dob_years` = 0. Необходимо проверить число уникальных значений методом value_counts() и оценить данные (вероятно таким образом отмечали отсутствие данных по возрасту)

**Вывод**

В каждой строке таблицы — информация о клиенте, взявшем кредит в банке. В столбцах находится информация о семейном положении клиента, количестве детей, доходах и т.д. В последнем столбце находится информация о цели получения кредита.

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, дубликаты с учетом регистра и аномальные значения

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Предобработка данных

### Обработка пропусков

In [4]:
credit_clients[credit_clients['days_employed'].isna()].count()

children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64

Оценивая величину пропущенных данных, можно увидеть, что данные в столбце `days_employed` отсутствуют ровно в тех же строках, что и в столбце `total_income`.

Оценим записи, в которых отсутствуют данные в этих столбцах

In [5]:
credit_clients[credit_clients['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


По таблице видно, что закономерность в наличии пропуска данных в столбцах `days_employed` и `total_income` и данными из других столбцов отсутствует. 

Скорее всего эта сводная таблица была собрана из разных источников. В некоторых информация о доходах и стаже была, а в некоторых нет. Возможно также некорректное заполнение части записей (вероятно для приведения данных из строкового типа в вещественный тип использовался метод to_numeric() с параметром errors = `coerce`, поэтому некорректные значения принудительно заменились на NaN).

В `days_employed` пропуски не важны для целей исследования, поэтому можно их заполнить явными обозначениями (заполним нулями для удобства дальнейшего преобразования в целочисленный тип данных).

Но пропуски в `total_income` могут помешать при проверке гипотезы  о влиянии уровня дохода на факт погашения кредита в срок. На практике было бы правильно установить причину пропусков и восстановить данные, связавшись с их составителями, но в учебном проекте такой возможности нет, поэтому придётся:
* заполнить эти пропуски медианой по соответствующим уровням дохода,
* оценить, насколько они повредят расчётам.

Будем применять именно медиану, поскольку среднее арифметическое для данных, у которых возможны большие выбросы (характерно для уровня дохода), применять не желательно. 

Начнем со столбца `days_employed`. Можно заметить, что почти все значения в столбце отрицательные. Возможная причина появления таких данных - это перевод строки в число вместе со знаком "тире" или ошибки при сохранении данных в таблицу.

Возьмём значения столбца по модулю, чтобы получить корректные значения

In [6]:
credit_clients['days_employed'] = credit_clients['days_employed'].abs()
# Проверка
credit_clients[credit_clients['days_employed'] < 0]['days_employed'].count()

0

Теперь рассмотрим аномально большие значения в том же столбце. Выведем на экран записи со стажем больше 20000 дней

In [7]:
display(credit_clients[credit_clients['days_employed'] > 20000].head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


In [8]:
credit_clients[credit_clients['days_employed'] > 20000]['income_type'].value_counts()

пенсионер      3443
безработный       2
Name: income_type, dtype: int64

Можно заметить, что все записи с таким большим значением стажа относятся либо к пенсионерам, либо к безработным. Возможно, ошибки связаны с тем, что здесь стаж указан не в днях, а в часах, поэтому для перевода в дни их нужна поделить на 24, а возможно - это ошибки, полученные при создании таблицы.

В рамках учебного задания точно установить причину таких аномальных значений невозможно, поэтому просто заменим их вместе с пропущенными значениями на характерные нулевые значения, так как эти данные не используются в исследовании.

In [9]:
credit_clients['days_employed'] = credit_clients['days_employed'].fillna(0)
credit_clients.loc[credit_clients.days_employed > 20000, 'days_employed'] = 0
# Проверка
credit_clients[credit_clients['days_employed'] > 20000]['days_employed'].count()

0

Перейдём к столбцу `total_income`. Как уже было указано выше заполним пропуски медианой по соответствующим уровням дохода.

In [10]:
try:
    credit_clients['total_income'] = credit_clients.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))
except:
    print("Приведите все значения в столбце 'total_income' к вещественному типу для вычисления медианы")
credit_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Я обнаружил пропуски в столбцах `days_employed` и `total_income`. 

Возможная причина появления пропуска в данных - это их отсутствие этих данных в определенных источниках или некорректное заполнение части записей (вероятно для приведения данных из строкового типа в вещественный тип использовался метод to_numeric() с параметром errors = `coerce`, поэтому некорректные значения принудительно заменились на NaN).

Столбец `days_employed` был заполнен аномальными отрицательными и очень большими значениями. Я домножил все отрицательные значения на -1, а аномально большие значения заменил нулевыми вместе с пропусками, так как значения из этого столбца не важны для целей исследования.

В `total_income` пропуски влияют на цель исследования, поэтому в этом столбце пропуски были заполнены характерными значениями выборки по разным `income_type` - медианами.


### Замена типа данных

В этом подпункте заменим тип данных на целочисленный в столбцах `days_employed` и `total_income`. Оптимизируем хранение данных во всех других столбцах. Заодно оценим количество данных с аномальными значениями в столбцах `children` и `dob_years` и при возможности удалим их

Опишем функцию, которая поможет нам определять сколько в памяти занимает DataFrame, чтобы каждый раз не вызывать info()

In [11]:
def mem_usage(pandas_obj):
    usage_b = pandas_obj.memory_usage(deep=True).sum()
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)
credit_clients.info()
print(mem_usage(credit_clients))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
11.68 MB


Преобразуем тип данных в столбцах `days_employed` и `total_income` на целочисленный

In [12]:
credit_clients['days_employed'] = credit_clients['days_employed'].astype('int')
credit_clients['total_income'] = credit_clients['total_income'].astype('int')

Для правильной категоризации данных сразу удалим дубликаты в категориях образования с учетом регистра

Рассмотрим уникальные значения по разным столбцам

In [13]:
display(credit_clients['education'].value_counts().to_frame())

,education
среднее,13750
высшее,4718
СРЕДНЕЕ,772
Среднее,711
неоконченное высшее,668
ВЫСШЕЕ,274
Высшее,268
начальное,250
Неоконченное высшее,47
НЕОКОНЧЕННОЕ ВЫСШЕЕ,29


В столбце `education` приведем все строки к нижнему регистру, чтобы избавить от дубликатов в категориях.

In [14]:
credit_clients['education'] = credit_clients['education'].str.lower()
# Проверка
display(credit_clients['education'].value_counts().to_frame())

,education
среднее,15233
высшее,5260
неоконченное высшее,744
начальное,282
ученая степень,6


In [15]:
education_dict = credit_clients[['education_id', 'education']]
# создание словаря, связывающего уровень образования с его id
education_dict = education_dict.drop_duplicates().reset_index(drop=True).sort_values('education_id')
display(education_dict)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [16]:
family_status_dict = credit_clients[['family_status_id', 'family_status']]
# создание словаря, связывающего семейное положение с его id
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True).sort_values('family_status_id')
display(family_status_dict)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Оптимизируем хранение информации в таблице.

In [17]:
credit_clients_int = credit_clients.select_dtypes(include=['int'])
converted_int = credit_clients_int.apply(pd.to_numeric,downcast='integer')
credit_clients[converted_int.columns] = converted_int

Изменим тип данных столбцев с `object` на `category`. Для столбцов `education` и `family_status` изменим порядок категории, чтобы соответствовать `education_id` и `family_status_id`.

In [18]:
credit_clients['gender'] = credit_clients['gender'].astype('category')
credit_clients['income_type'] = credit_clients['income_type'].astype('category')

credit_clients['education'] = credit_clients['education'].astype('category')
credit_clients['family_status'] = credit_clients['family_status'].astype('category')

# Изменение порядка категорий для соответствия столбцу education_id и family_status_id
credit_clients['education'] = credit_clients['education'].cat.reorder_categories(education_dict['education'], ordered=True)
credit_clients['family_status'] = credit_clients['family_status'].cat.reorder_categories(family_status_dict['family_status'], ordered=True)

# Удаление не нужных теперь столбцов education_id и family_status_id
credit_clients = credit_clients.drop('education_id', axis=1)
credit_clients = credit_clients.drop('family_status_id', axis=1)

print(mem_usage(credit_clients))

2.85 MB


In [19]:
credit_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 10 columns):
children         21525 non-null int8
days_employed    21525 non-null int16
dob_years        21525 non-null int8
education        21525 non-null category
family_status    21525 non-null category
gender           21525 non-null category
income_type      21525 non-null category
debt             21525 non-null int8
total_income     21525 non-null int32
purpose          21525 non-null object
dtypes: category(4), int16(1), int32(1), int8(3), object(1)
memory usage: 442.4+ KB


Оценим уникальные значения в столбце `children` и их количество

In [20]:
credit_clients['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Из общего ряда выбиваются значения -1 и 20. Как уже говорилось выше, значение -1 может означать отсутствие данных по количеству детей, а вот значение 20 скорее всего является случайной ошибкой. В любом случае, истинный смысл этих данных установить невозможно, а их суммарное количество не превышает 1% от общего числа записей, поэтому эти данные можно удалить (эти данные непосредственно участвуют в исследовании)

In [21]:
credit_clients = credit_clients[(credit_clients['children'] <= 5) & (credit_clients['children'] >= 0)]
credit_clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21402 entries, 0 to 21524
Data columns (total 10 columns):
children         21402 non-null int8
days_employed    21402 non-null int16
dob_years        21402 non-null int8
education        21402 non-null category
family_status    21402 non-null category
gender           21402 non-null category
income_type      21402 non-null category
debt             21402 non-null int8
total_income     21402 non-null int32
purpose          21402 non-null object
dtypes: category(4), int16(1), int32(1), int8(3), object(1)
memory usage: 607.0+ KB


Сделаем тоже самое со столбцом `dob_years`

In [22]:
credit_clients['dob_years'].value_counts()

35    614
40    603
41    603
34    597
38    595
42    592
33    577
39    572
31    556
36    553
29    543
44    543
30    536
48    536
37    531
43    510
50    509
32    506
49    505
28    501
45    494
27    490
52    483
56    482
47    480
54    476
46    469
58    461
57    457
53    457
51    446
55    441
59    441
26    406
60    376
25    356
61    353
62    351
63    268
64    263
24    263
23    252
65    194
66    183
22    183
67    167
21    110
0     100
68     99
69     83
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

В этом столбце выделяется значение 0. Вероятно, так заполняли записи, в которых был неизвестен возраст клиента, либо же опять случайные ошибки. В этом столбце было принято решение эти строки не трогать, так как в них имеются неповрежденные полезные для целей исследования данные, когда как ошибка кроется в возрасте клиента, который по существу не относятся к исследуемой проблеме.

**Вывод**

Для преобразования типов был использован метод `astype()` с аргументом `'int'`. Для обработки ошибок применяется конструкция try-except.

Также были удалены строки с аномальными значениями по количеству детей.

### Обработка дубликатов

Рассмотрим уникальные значения по разным столбцам

In [23]:
display(credit_clients['family_status'].value_counts().to_frame())

,family_status
женат / замужем,12302
гражданский брак,4160
Не женат / не замужем,2799
в разводе,1189
вдовец / вдова,952


In [24]:
display(credit_clients['gender'].value_counts().to_frame())

,gender
F,14154
M,7247
XNA,1


In [25]:
display(credit_clients['income_type'].value_counts().to_frame())

,income_type
сотрудник,11050
компаньон,5054
пенсионер,3839
госслужащий,1453
предприниматель,2
безработный,2
студент,1
в декрете,1


Определим количество явных дубликатов после приведения к нижнему регистру и избавимся от них

In [26]:
credit_clients.duplicated().sum()

71

In [27]:
credit_clients = credit_clients.drop_duplicates()
credit_clients.duplicated().sum()

0

**Вывод**

Были обнаружены явные дубликаты в данных. Они были удалены

Были обнаружены дубликаты в столбце `education`. Значения, относящиеся к одинаковым уровням образования, были записаны с помощью разных регистров. Для правильного учета этих данных они были преведены к нижнему регистру.

### Лемматизация

В столбце `purpose` записаны цели получения кредита. Рассмотрим уникальные значения в этом столбце

In [28]:
credit_clients['purpose'].value_counts()

свадьба                                   790
на проведение свадьбы                     763
сыграть свадьбу                           760
операции с недвижимостью                  672
покупка коммерческой недвижимости         658
покупка жилья для сдачи                   649
операции с жильем                         647
операции с коммерческой недвижимостью     645
жилье                                     641
покупка жилья                             640
покупка жилья для семьи                   637
недвижимость                              631
строительство собственной недвижимости    628
операции со своей недвижимостью           623
строительство жилой недвижимости          620
покупка своего жилья                      619
строительство недвижимости                619
покупка недвижимости                      616
ремонт жилью                              604
покупка жилой недвижимости                602
на покупку своего автомобиля              504
заняться высшим образованием      

По значениям в таблице явно видны дубликаты. Для правильной категоризации и оценки влияния цели кредита на наличие задолженности стоит выделить какое-то одно слово характеризующее цель кредита. Для этого лемматизируем цели кредита, вручную составим категории на основе таблицы сверху и присвоим записям соответствующие категории по поиску в леммах по целям

In [29]:
categories = ['жилье', 'недвижимость', 'свадьба', 'автомобиль', 'образование']

Создадим вспомогательный столбец `lemmatized_purpose` с леммами по целям кредита на время.

In [30]:
m = Mystem() 
credit_clients['lemmatized_purpose'] = credit_clients['purpose'].apply(m.lemmatize)
credit_clients.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,lemmatized_purpose
0,1,8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"
5,0,926,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]"
6,0,2879,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]"
7,0,152,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,"[образование, \n]"
8,2,6929,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]"
9,0,2188,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]"


In [31]:
def categorise_purpose(purpose):
    """
    Возвращает категорию из списка categories по цели кредита, используя лемматизированную цель кредита
    """
    for category in categories:
        if category in purpose:
            return category

In [32]:
credit_clients['category_purpose'] = credit_clients['lemmatized_purpose'].apply(categorise_purpose)
# удаляем вспомогательный столбец
credit_clients = credit_clients.drop('lemmatized_purpose', axis=1)
# заменим категорию 'недвижимость' на категорию 'жилье', так как это фактически одно и тоже
credit_clients['category_purpose'] = credit_clients['category_purpose'].replace('недвижимость', 'жилье')
credit_clients.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,category_purpose
0,1,8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование
4,0,0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,жилье


Проверим заполненность столбца `category_purpose`

In [33]:
credit_clients.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21331 entries, 0 to 21524
Data columns (total 11 columns):
children            21331 non-null int8
days_employed       21331 non-null int16
dob_years           21331 non-null int8
education           21331 non-null category
family_status       21331 non-null category
gender              21331 non-null category
income_type         21331 non-null category
debt                21331 non-null int8
total_income        21331 non-null int32
purpose             21331 non-null object
category_purpose    21331 non-null object
dtypes: category(4), int16(1), int32(1), int8(3), object(2)
memory usage: 771.6+ KB


In [34]:
credit_clients['category_purpose'].value_counts()

жилье          10751
автомобиль      4279
образование     3988
свадьба         2313
Name: category_purpose, dtype: int64

**Вывод**

Лемматизация целей кредита была проведена с помощью библиотеки `PyMystem` с функцией `lemmatize` и метода `apply()` для применения этой функции ко всем значениям столбца. Для категоризации по целям кредита был составлен список с целями кредита и произведен поиск по уже лемматизированным целям.

### Категоризация данных

Категоризируем данные согласно целям исследования. Создадим функцию, на вход которой будет подаваться количество детей, а возвращаться категория. Именно такие категории были выбраны так как проверяемой гипотезой считается проверка влияния именно наличия детей, а не их количества.

In [35]:
def child_cat(children):
    """
    Возвращает группу по количеству детей children, согласно разбиению:
    - 'есть дети', если children > 0;
    - 'нет детей', если детей нет
    """
    
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'

In [36]:
credit_clients['children_cat'] = credit_clients['children'].apply(child_cat)
credit_clients.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,category_purpose,children_cat
0,1,8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,жилье,есть дети
1,1,4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети
2,0,5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,жилье,нет детей
3,3,4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование,есть дети
4,0,0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,926,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,жилье,нет детей
6,0,2879,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,жилье,нет детей
7,0,152,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,свадьба,есть дети
9,0,2188,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,жилье,нет детей


Категоризируем данные по уровню дохода. Создадим функцию, на вход которой будет подаваться ежемесячный доход, а возвращаться категория.

Федеральная служба государственной статистики разделяет уровень жизни россиян в зависимости от доходов на следующие категории:
1. крайняя нищета (доходы ниже прожиточного минимума — до 7-8 тыс.р.)
2. нищета (доходы от одного до двух прожиточных минимума — от 8 до 12 тыс.р.)
3. бедность (доходы от 12 до 20 тысяч рублей в месяц)
4. выше бедности (доходы от 20 до 30 тысяч рублей в месяц)
5. средний достаток — (доходы от 30 до 60 тысяч рублей в месяц)
6. состоятельные — (доходы от 60 до 90 тысяч рублей в месяц)
7. богатые — (доходы от 90 тысяч рублей в месяц)
8. сверхбогатые — (доходы свыше 150 тысяч рублей в месяц)

Укрупним категории (одновременно соотносим категории с реальностью):
1. низкий (доходы до 60 тысяч рублей в месяц)
2. средний (доходы от 60 до 120 тысяч рублей в месяц)
3. высокий (доходы от 120 до 200 тысяч рублей в месяц)
4. сверхвысокий (доходы от 200 тысяч рублей в месяц)

In [37]:
def total_income_cat(total_income):
    """
    Возвращает категорию по уровню дохода total_income, согласно разбиению
    - 'низкий', если total_income <= 60000;
    - 'средний', если 60000 < total_income <= 120000;
    - 'высокий', если 120000 < total_income <= 200000
    - 'сверхвысокий', если 200000 < total_income
    """
    if total_income <= 60000:
        return 'низкий'
    elif 60000 < total_income <= 120000:
        return 'средний'
    elif 120000 < total_income <= 200000:
        return 'высокий'
    else:
        return 'сверхвысокий'

In [38]:
credit_clients['total_income_cat'] = credit_clients['total_income'].apply(total_income_cat)
credit_clients.head(10)

,children,days_employed,dob_years,education,family_status,gender,income_type,debt,total_income,purpose,category_purpose,children_cat,total_income_cat
0,1,8437,42,высшее,женат / замужем,F,сотрудник,0,253875,покупка жилья,жилье,есть дети,сверхвысокий
1,1,4024,36,среднее,женат / замужем,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,есть дети,средний
2,0,5623,33,среднее,женат / замужем,M,сотрудник,0,145885,покупка жилья,жилье,нет детей,высокий
3,3,4124,32,среднее,женат / замужем,M,сотрудник,0,267628,дополнительное образование,образование,есть дети,сверхвысокий
4,0,0,53,среднее,гражданский брак,F,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,высокий
5,0,926,27,высшее,гражданский брак,M,компаньон,0,255763,покупка жилья,жилье,нет детей,сверхвысокий
6,0,2879,43,высшее,женат / замужем,F,компаньон,0,240525,операции с жильем,жилье,нет детей,сверхвысокий
7,0,152,50,среднее,женат / замужем,M,сотрудник,0,135823,образование,образование,нет детей,высокий
8,2,6929,35,высшее,гражданский брак,F,сотрудник,0,95856,на проведение свадьбы,свадьба,есть дети,средний
9,0,2188,41,среднее,женат / замужем,M,сотрудник,0,144425,покупка жилья для семьи,жилье,нет детей,высокий


**Вывод**

Была проведена категоризация данных согласно целям исследования. Были выделены словари, соединяющие id с описаниями.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [39]:
# Группировка по наличию детей с числом записей с соответствующей категорией
grouped = credit_clients.groupby('children_cat')['debt'].count().to_frame().sort_values(by='children_cat')
# Группировка по наличию детей со средним арифметическим по столбцу с наличием долга 
# (фактически отношение записей с долгом ко всем по данной категории)
children_pivot = credit_clients.groupby('children_cat')['debt'].mean().to_frame().sort_values(by='children_cat')
# Слияние таблиц в одну
children_pivot = grouped.merge(children_pivot, on='children_cat')
children_pivot = children_pivot.rename(columns={'debt_x':'count',
                                                'debt_y':'debt_ratio'
})

display(children_pivot)

,count,debt_ratio
children_cat,,
есть дети,7240,0.092403
нет детей,14091,0.075438


**Вывод**

Клиенты, имеющие детей, чаще не выплачивают кредит во время.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [40]:
grouped = credit_clients.groupby('family_status')['debt'].count().to_frame().sort_values(by='family_status')
family_status_pivot = credit_clients.groupby('family_status')['debt'].mean().to_frame().sort_values(by='family_status')
family_status_pivot = grouped.merge(family_status_pivot, on='family_status')
family_status_pivot = family_status_pivot.rename(columns={'debt_x':'count',
                                                          'debt_y':'debt_ratio'
})
display(family_status_pivot)

,count,debt_ratio
family_status,,
женат / замужем,12261,0.075606
гражданский брак,4134,0.093130
вдовец / вдова,951,0.066246
в разводе,1189,0.070648
Не женат / не замужем,2796,0.097639


**Вывод**

Люди не в браке и в гражданском браке чаще не платят кредит вовремя

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [41]:
grouped = credit_clients.groupby('total_income_cat')['debt'].count().to_frame().sort_values(by='total_income_cat')
total_income_pivot = credit_clients.groupby('total_income_cat')['debt'].mean().to_frame().sort_values(by='total_income_cat')
total_income_pivot = grouped.merge(total_income_pivot, on='total_income_cat')
total_income_pivot = total_income_pivot.rename(columns={'debt_x':'count',
                                              'debt_y':'debt_ratio'
})

display(total_income_pivot)

,count,debt_ratio
total_income_cat,,
высокий,9096,0.086961
низкий,801,0.061174
сверхвысокий,5039,0.070649
средний,6395,0.083815


**Вывод**

Люди, имеющие низкий достаток чаще возвращают кредит в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
grouped = credit_clients.groupby('category_purpose')['debt'].count().to_frame().sort_values(by='category_purpose')
purpose_pivot = credit_clients.groupby('category_purpose')['debt'].mean().to_frame().sort_values(by='category_purpose')
purpose_pivot = grouped.merge(purpose_pivot, on='category_purpose')
purpose_pivot = purpose_pivot.rename(columns={'debt_x':'count',
                                              'debt_y':'debt_ratio'
})

display(purpose_pivot)

,count,debt_ratio
category_purpose,,
автомобиль,4279,0.093480
жилье,10751,0.072551
образование,3988,0.092528
свадьба,2313,0.079118


**Вывод**

Люди, берущие кредит на жильё и на свадьбу чаще выплачивают кредит в срок.

## Шаг 4. Общий вывод

Семеное положение и количество детей влияет на факт погашения кредита в срок:
1. Семейное положение влияет на факт погашения кредита в срок. Люди, которые официально не состоят в браке, чаще не выплачивают кредит в срок
2. Наличие детей влияет на факт погашения кредита в срок. Люди, имеют детей, чаще не выплачивают кредит в срок
3. Уровень дохода влияет на факт погашения кредита в срок. Люди с низким достатком чаще выплачивают кредит в срок
4. Цель кредита влияет на на факт погашения кредита в срок. Люди, берущие кредит на жилье и свадьбу чаще выплачивают кредит в срок

Рассмотрим сводные таблицы по разным категориям

In [43]:
total_children_pivot = credit_clients.pivot_table(index=['category_purpose'], columns='children_cat', values='debt', aggfunc='mean')
display(total_children_pivot)

children_cat,есть дети,нет детей
category_purpose,,
автомобиль,0.109484,0.085413
жилье,0.082676,0.067289
образование,0.104012,0.086677
свадьба,0.086845,0.075163


In [44]:
purpose_children_pivot = credit_clients.pivot_table(index=['total_income_cat'], columns='children_cat', values='debt', aggfunc='mean')
display(purpose_children_pivot)

children_cat,есть дети,нет детей
total_income_cat,,
высокий,0.098553,0.080527
низкий,0.086364,0.051635
сверхвысокий,0.075800,0.067833
средний,0.097892,0.077447


In [45]:
family_purpose_pivot = credit_clients.pivot_table(index=['family_status'], columns='category_purpose', values='debt', aggfunc='mean')
display(family_purpose_pivot)

category_purpose,автомобиль,жилье,образование,свадьба
family_status,,,,
женат / замужем,0.083180,0.069727,0.083528,NaN
гражданский брак,0.118605,0.092105,0.148883,0.079118
вдовец / вдова,0.091743,0.052336,0.075758,NaN
в разводе,0.075269,0.068452,0.071429,NaN
Не женат / не замужем,0.129134,0.081337,0.107826,NaN
